### FLINK-25728 Reproduction
---  
#### Load Session Data

In [ ]:
import scripts.session as session

session_data = session.load()

#### Copy the code repo to node


In [ ]:
from chi import ssh
import scripts.ssh_helper as ssh_helper

In [ ]:
# put the trovi to node

with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('sudo mkdir -p /mnt/flink-exp; sudo chown $USER -R /mnt; cd /mnt/flink-exp; rm -rf /mnt/flink-exp/flink_bug_reproduce; mkdir -p /mnt/spark-exp; cd /mnt/flink-exp; git clone https://github.com/wpc009/flink.git; cd flink; git checkout FLINK-25728')

#### Set up the Flink evironment

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/flink-exp/flink; mvn clean compile -B -fn -DskipTests 2>&1 | tee build.log')

You can see from the build log that the project fails to be built. Apart from build failure of other parts of the project, the most critical error of building the project occurs on the project "Test Utils" which prevents us from running any tests using methods from "Test Utils", including the test of this bug: MultipleInputStreamMemoryIssueTest and NaryInputTest. But this problem doesn't happen on the latest release of Flink system, because the dependencies of different projects are different.

For example, if we run the test MultipleInputStreamMemoryIssueTest, the test will fail because it cannot resolve dependencies:

In [ ]:
with ssh.Remote(session_data["floating_ip"]) as conn:
    conn.run('cd /mnt/flink-exp/flink/flink-end-to-end-tests/flink-datastream-multiple-input-test; mvn test -Dtest=org.apache.flink.streaming.tests.MultipleInputStreamMemoryIssueTest -B -fn 2>&1 | tee single_test_multiple_input_build.log')